In [116]:
from io import StringIO
import pandas as pd
import requests
import lxml.html

In [117]:
# URL of the webpage
url = "https://bookretreats.com/blog/best-places-to-visit-in-usa/#h-full-dataset"

# Send an HTTP GET request to fetch the page
response = requests.get(url)

# Wrap the HTML content in a StringIO object
html_content = StringIO(response.text)

# Parse all tables found in the HTML content
tables = pd.read_html(html_content)

# Assuming the table of interest is the first one
df = tables[2]

# Assuming the table of interest is the first one
df = tables[2]

In [118]:
df.head()

,Rank,U.S Attraction,Location,Popularity/1,Experience/2,Affordability/1,Safety Score/1,Total
0,1,Central Park,"New York, NY",0.98,1.85,0.73,0.52,4.075
1,2,National Mall,"Washington, DC",0.69,1.84,0.92,0.55,3.994
2,3,Balboa Park,"San Diego, CA",0.55,1.86,0.89,0.49,3.795
3,4,Smithsonian National Museum of Natural History,"Washington, D.C.",0.50,1.82,0.91,0.55,3.775
4,5,Times Square,"New York, NY",0.84,1.64,0.73,0.52,3.730


In [119]:
from sklearn.preprocessing import StandardScaler
# Rename columns to more convenient names
df = df.rename(columns={
    "U.S Attraction": "Attraction",
    "Popularity/1": "Popularity",
    "Experience/2": "Experience",
    "Affordability/1": "Affordability",
    "Safety Score/1": "Safety"
})

In [120]:
df.head()

,Rank,Attraction,Location,Popularity,Experience,Affordability,Safety,Total
0,1,Central Park,"New York, NY",0.98,1.85,0.73,0.52,4.075
1,2,National Mall,"Washington, DC",0.69,1.84,0.92,0.55,3.994
2,3,Balboa Park,"San Diego, CA",0.55,1.86,0.89,0.49,3.795
3,4,Smithsonian National Museum of Natural History,"Washington, D.C.",0.50,1.82,0.91,0.55,3.775
4,5,Times Square,"New York, NY",0.84,1.64,0.73,0.52,3.730


In [121]:
def process_location(loc):
    # Check for Washington
    if loc.strip() in ["Washington, DC", "Washington, D.C."]:
        return pd.Series(["Washington, D.C.", "VA"])
    # If there is a comma, split by it
    if "," in loc:
        parts = [p.strip() for p in loc.split(",", 1)]
        return pd.Series(parts)
    # Otherwise, return the whole value as City and NaN for State
    return pd.Series([loc.strip(), None])

# Apply the helper function to the Location column for rows that are not Washington
df[["City", "State"]] = df["Location"].apply(process_location)

# Remove any extra whitespace
df['City'] = df['City'].str.strip()
df['State'] = df['State'].str.strip()

# Drop the original 'location' column
df = df.drop(columns=['Location'])

df.head()

,Rank,Attraction,Popularity,Experience,Affordability,Safety,Total,City,State
0,1,Central Park,0.98,1.85,0.73,0.52,4.075,New York,NY
1,2,National Mall,0.69,1.84,0.92,0.55,3.994,"Washington, D.C.",VA
2,3,Balboa Park,0.55,1.86,0.89,0.49,3.795,San Diego,CA
3,4,Smithsonian National Museum of Natural History,0.50,1.82,0.91,0.55,3.775,"Washington, D.C.",VA
4,5,Times Square,0.84,1.64,0.73,0.52,3.730,New York,NY


In [122]:
df.to_csv(
    r"C:\Users\isabe\PycharmProjects\GSB 520 ETL\Clean Data\tourism_cleaned.csv", index=False)